In [1]:
import pandas as pd
import requests

In [2]:
url = 'https://python.zgulde.net/api/v1/items'
response = requests.get(url)
response.text
data = response.json()
data

{'payload': {'items': [{'item_brand': 'Riceland',
    'item_id': 1,
    'item_name': 'Riceland American Jazmine Rice',
    'item_price': 0.84,
    'item_upc12': '35200264013',
    'item_upc14': '35200264013'},
   {'item_brand': 'Caress',
    'item_id': 2,
    'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
    'item_price': 6.44,
    'item_upc12': '11111065925',
    'item_upc14': '11111065925'},
   {'item_brand': 'Earths Best',
    'item_id': 3,
    'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
    'item_price': 2.43,
    'item_upc12': '23923330139',
    'item_upc14': '23923330139'},
   {'item_brand': 'Boars Head',
    'item_id': 4,
    'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
    'item_price': 3.14,
    'item_upc12': '208528800007',
    'item_upc14': '208528800007'},
   {'item_brand': 'Back To Nature',
    'item_id': 5,
    'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
    'item_price':

In [3]:
page_items = data['payload']['items']
page_items

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'},
 {'item_brand': 'Earths Best',
  'item_id': 3,
  'item_name': 'Earths Best Organic Fruit Yogurt Smoothie Mixed Berry',
  'item_price': 2.43,
  'item_upc12': '23923330139',
  'item_upc14': '23923330139'},
 {'item_brand': 'Boars Head',
  'item_id': 4,
  'item_name': 'Boars Head Sliced White American Cheese - 120 Ct',
  'item_price': 3.14,
  'item_upc12': '208528800007',
  'item_upc14': '208528800007'},
 {'item_brand': 'Back To Nature',
  'item_id': 5,
  'item_name': 'Back To Nature Gluten Free White Cheddar Rice Thin Crackers',
  'item_price': 2.61,
  'item_upc12': '759283100036',
  'item_upc14': '759283100036'},
 {'i

In [4]:
data['payload']['max_page']


3

In [5]:
items = pd.DataFrame(page_items)

In [6]:
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [7]:
# follow this procedure for stores and then sales

In [8]:
# set up a base url string
base_url = 'https://python.zgulde.net'
# tack on an endpoint
response = requests.get(base_url + '/api/v1/stores')

In [9]:
response.json()['payload']['max_page']

1

In [10]:
stores = pd.DataFrame(response.json()['payload']['stores'])

In [11]:
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [12]:
#Make a request of the first page of data
response_sales = requests.get('https://python.zgulde.net/api/v1/sales')

data_sales = response_sales.json()

data_sales.keys()

dict_keys(['payload', 'status'])

In [13]:
#Take a look at the keys
data_sales['payload'].keys()

dict_keys(['max_page', 'next_page', 'page', 'previous_page', 'sales'])

In [14]:
#What's the max page in my dataset?
data_sales['payload']['max_page'], type(data_sales['payload']['max_page'])

(183, int)

In [15]:
#Save the max page integer to a variable
max_page_sales = data_sales['payload']['max_page']

In [16]:
#Loop through the pages and concatenate the request results to my dataframe
sales_url = 'https://python.zgulde.net/api/v1/sales?page='

sales = pd.DataFrame(data_sales['payload']['sales'])

for i in range(2, max_page_sales + 1):
    response = requests.get(sales_url + str(i))
    json = response.json()
    df = pd.DataFrame(json['payload']['sales'])
    sales = pd.concat([sales, df])
    
sales.shape    

(913000, 5)

In [17]:
#Reset the index
sales.reset_index(inplace=True)

In [18]:
#Store the results of my requests to csv files
items.to_csv('items.csv')
stores.to_csv('stores.csv')
sales.to_csv('sales.csv')

In [19]:
#Left merge items onto sales
super_df = sales.merge(items, how='left',
                       left_on='item', right_on='item_id')
super_df.head()

,index,item,sale_amount,sale_date,sale_id,store,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [20]:
#Left merge stores onto my super dataframe
super_df = super_df.merge(stores, how='left',
                          left_on='store', right_on='store_id')
super_df.head()

,index,item,sale_amount,sale_date,sale_id,store,item_brand,item_id,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_id,store_state,store_zipcode
0,0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
2,2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
3,3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
4,4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1,Riceland,1.0,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [21]:
#Drop unnecessary columns from my dataframe
super_df.drop(columns=['index_x', 'item', 'sale_id', 'store', 'level_0',
                       'index_y', 'item_id', 'item_upc12', 'item_upc14',
                       'store_id'], inplace=True)
super_df.head()

KeyError: "['index_x', 'level_0', 'index_y'] not found in axis"

In [ ]:
#Save my dataframe
super_df.to_csv('merged_df.csv')

In [ ]:
#Read a csv file from a url
power_df = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

In [ ]:
#Make sure I captured data
power_df.head()

In [ ]:
#Save to csv
power_df.to_csv('power.csv')

In [ ]:
#Define a function to read all items data and return a dataframe
def get_items():
    
    base_url = 'https://python.zgulde.net'
    
    response = requests.get('https://python.zgulde.net/api/v1/items')
    
    data = response.json()
    
    items = pd.DataFrame(data['payload']['items'])
    
    next_page = data['payload']['next_page']
    
    while next_page != None:
        
        response = requests.get(base_url + next_page)
        
        data = response.json()
        
        next_page = data['payload']['next_page']
        
        new_items = pd.DataFrame(data['payload']['items'])
        
        items = pd.concat([items, new_items])
    
    items.reset_index(drop=True, inplace=True)
    
    #items.to_csv('items.csv')
    
    return items

In [ ]:
#Execute the function and look at results
items_test = get_items()
items_test.head()

In [ ]:
#Make sure I grab the expected number of items
items_test.shape